In [63]:
pip install flask flask-sqlalchemy flask-migrate flask-mail pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [65]:
###Configuration de Flask et SQLAlchemy

from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
from flask_mail import Mail, Message
import pandas as pd
import datetime

app = Flask(__name__)

# Configuration of the SQlite database 
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.secret_key = 'supersecretkey'

# Configuration of mails (e.i Gmail)
app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'ton_email@gmail.com'
app.config['MAIL_PASSWORD'] = 'ton_mot_de_passe'

db = SQLAlchemy(app)
mail = Mail(app)


In [67]:
### model of data base, adding: benevolence/volunteer , Mission/task , Inscription/register with a following of the activities they are doing

class Benevole(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nom = db.Column(db.String(100), nullable=False)
    prenom = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    telephone = db.Column(db.String(20), nullable=False)
    dernieres_activites = db.Column(db.DateTime, default=datetime.datetime.utcnow)

    inscriptions = db.relationship('Inscription', backref='benevole', lazy=True)

class Mission(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nom = db.Column(db.String(100), nullable=False)
    description = db.Column(db.Text, nullable=False)
    nb_places = db.Column(db.Integer, nullable=False)

    inscriptions = db.relationship('Inscription', backref='mission', lazy=True)

class Inscription(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    benevole_id = db.Column(db.Integer, db.ForeignKey('benevole.id'), nullable=False)
    mission_id = db.Column(db.Integer, db.ForeignKey('mission.id'), nullable=False)
    date_inscription = db.Column(db.DateTime, default=datetime.datetime.utcnow)


In [ ]:
import os
from werkzeug.utils import secure_filename

UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file.filename.endswith('.xlsx'):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)
            importer_benevoles(filepath)
            flash('Données importées avec succès !', 'success')
        else:
            flash('Format invalide. Veuillez uploader un fichier .xlsx', 'danger')

    return render_template('upload.html')

def importer_benevoles(filepath):
    df = pd.read_excel(filepath)
    for _, row in df.iterrows():
        benevole = Benevole(
            nom=row['Nom'], prenom=row['Prenom'], email=row['Email'], telephone=row['Telephone']
        )
        db.session.add(benevole)
    db.session.commit()
